In [1]:
import collections
import functools
import glob
import pickle
import itertools
import json
import os
import random
import sys
import numpy as np
import tqdm
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# If domainbed is a custom module, you might need to adjust its imports or setup
from domainbed import datasets
from domainbed import algorithms
from domainbed.lib import misc, reporting
from domainbed import model_selection
from domainbed.lib.query import Q
import warnings


/Users/yuenc2/anaconda3/envs/fishr/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/yuenc2/anaconda3/envs/fishr/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <2D1B8D5C-7891-3680-9CF9-F771AE880676> /Users/yuenc2/anaconda3/envs/fishr/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <8D9916BF-BDB2-35FE-B43C-E28A9CA7D22C> /Users/yuenc2/anaconda3/envs/fishr/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
def count_subdirectories_and_empty_ones(directory):
    num_subdirectories = 0
    num_empty_subdirectories = 0

    # Walk through all directories in the given directory
    for root, dirs, files in os.walk(directory):
        # Iterate over each directory in the current root
        for d in dirs:
            num_subdirectories += 1
            subdirectory_path = os.path.join(root, d)
            # Check if the directory is empty
            if not os.listdir(subdirectory_path):
                num_empty_subdirectories += 1

    return num_subdirectories, num_empty_subdirectories


# Specify the directory you want to inspect
# directory_path = 'results_vits_3600_32'
# directory_path = 'results_vits_terra_pacs'
# directory_path = 'results_vits_VLCS_ERM_Fishr'




directory_path_list = ['results_vits_hessian_real_bias', 'results_vits_hessian_bias','results_vits_coral']


for directory_path in directory_path_list:
    # Get the count of subdirectories and empty subdirectories
    subdirectories, empty_subdirectories = count_subdirectories_and_empty_ones(directory_path)
    print(f"Total subdirectories: {subdirectories}")
    print(f"Empty subdirectories: {empty_subdirectories} in {directory_path} directory")

# Get the count of subdirectories and empty subdirectories
# subdirectories, empty_subdirectories = count_subdirectories_and_empty_ones(directory_path)
# print(f"Total subdirectories: {subdirectories}")
# print(f"Empty subdirectories: {empty_subdirectories}")

Total subdirectories: 180
Empty subdirectories: 0 in results_vits_hessian_real_bias directory
Total subdirectories: 135
Empty subdirectories: 0 in results_vits_hessian_bias directory
Total subdirectories: 180
Empty subdirectories: 0 in results_vits_coral directory


In [3]:
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved
import itertools
import numpy as np

def get_test_records(records):
    """Given records with a common test env, get the test records (i.e. the
    records with *only* that single test env and no other test envs)"""
    return records.filter(lambda r: len(r['args']['test_envs']) == 1)

class SelectionMethod:
    """Abstract class whose subclasses implement strategies for model
    selection across hparams and timesteps."""

    def __init__(self):
        raise TypeError

    @classmethod
    def run_acc(self, run_records):
        """
        Given records from a run, return a {val_acc, test_acc} dict representing
        the best val-acc and corresponding test-acc for that run.
        """
        raise NotImplementedError

    @classmethod
    def hparams_accs(self, records):
        """
        Given all records from a single (dataset, algorithm, test env) pair,
        return a sorted list of (run_acc, records) tuples.
        """

        return (records.group('args.hparams_seed')
            .map(lambda _, run_records:
                (
                    self.run_acc(run_records),
                    run_records
                )
            ).filter(lambda x: x[0] is not None)
            .sorted(key=lambda x: x[0]['val_acc'])[::-1]
        )

    @classmethod
    def sweep_acc(self, records):
        """
        Given all records from a single (dataset, algorithm, test env) pair,
        return the mean test acc of the k runs with the top val accs.
        """
        _hparams_accs = self.hparams_accs(records)
        a = _hparams_accs
        # for i in range(len(a)):
        #     print(a[i][0]['val_acc'], a[i][0]['test_acc'])
        #     print(f"Hparams {(a[i][1][0]['hparams']['grad_alpha'], a[i][1][0]['hparams']['grad_alpha'],  a[i][1][0]['hparams']['penalty_anneal_iters'])}")
        # if a[0][1][0]['args']['algorithm'] == 'HessianAlignment':
        # if a[0][1][0]['args']['algorithm'] in ['HessianAlignment'] and a[0][1][0]['args']['dataset'] == 'PACS':
        # if a[0][1][0]['args']['algorithm'] in ['HessianAlignment'] and a[0][1][0]['args']['dataset'] == 'TerraIncognita':
        # if a[0][1][0]['args']['algorithm'] in ['HessianAlignment'] and a[0][1][0]['args']['dataset'] in ['VLCS','ColoredMNIST']:
        # if a[0][1][0]['args']['algorithm'] in ['HessianAlignment'] and a[0][1][0]['args']['dataset'] == 'ColoredMNIST':
        # if a[0][1][0]['args']['algorithm'] in ['Fishr'] and a[0][1][0]['args']['dataset'] == 'ColoredMNIST':
        #     print(f"Algorithm: {a[0][1][0]['args']['algorithm']}")
        #     print(f"Best val acc and test for {a[0][1][0]['args']['dataset']}, env {a[0][1][0]['args']['test_envs']}:", a[0][0]['val_acc'], a[0][0]['test_acc'], "Hparams", (a[0][1][0]['hparams']))
        #     print("Output dir", a[0][1][0]['args']['output_dir'])
        #    print(f"Best hyperparameters for {a[0][1][0]['args']['dataset']}, env {a[0][1][0]['args']['test_envs']}:", 
        #          f"grad_alpha: {a[0][1][0]['hparams']['grad_alpha']}, hess_beta {a[0][1][0]['hparams']['hess_beta']}, "
        #          f"anneal_iters: {a[0][1][0]['hparams']['penalty_anneal_iters']}",
        #          f"output_dir: {a[0][1][0]['args']['output_dir']}")

        if len(_hparams_accs):
            # breakpoint()
            return _hparams_accs[0][0]['test_acc']
        else:
            return None

class OracleSelectionMethod(SelectionMethod):
    """Like Selection method which picks argmax(test_out_acc) across all hparams
    and checkpoints, but instead of taking the argmax over all
    checkpoints, we pick the last checkpoint, i.e. no early stopping."""
    name = "test-domain validation set (oracle)"

    @classmethod
    def run_acc(self, run_records):
        run_records = run_records.filter(lambda r:
            len(r['args']['test_envs']) == 1)
        if not len(run_records):
            return None
        test_env = run_records[0]['args']['test_envs'][0]
        test_out_acc_key = 'env{}_out_acc'.format(test_env)
        test_in_acc_key = 'env{}_in_acc'.format(test_env)
        chosen_record = run_records.sorted(lambda r: r['step'])[-1]
        return {
            'val_acc':  chosen_record[test_out_acc_key],
            'test_acc': chosen_record[test_in_acc_key]
        }

class IIDAccuracySelectionMethod(SelectionMethod):
    """Picks argmax(mean(env_out_acc for env in train_envs))"""
    name = "training-domain validation set"

    @classmethod
    def _step_acc(self, record):
        """Given a single record, return a {val_acc, test_acc} dict."""
        test_env = record['args']['test_envs'][0]
        val_env_keys = []
        for i in itertools.count():
            if f'env{i}_out_acc' not in record:
                break
            if i != test_env:
                val_env_keys.append(f'env{i}_out_acc')
        test_in_acc_key = 'env{}_in_acc'.format(test_env)
        return {
            'val_acc': np.mean([record[key] for key in val_env_keys]),
            'test_acc': record[test_in_acc_key]
        }

    @classmethod
    def run_acc(self, run_records):
        test_records = get_test_records(run_records)
        if not len(test_records):
            return None

        index_of_max = test_records.map(self._step_acc).map(lambda x: x['val_acc'])._list.index(
            max(test_records.map(self._step_acc).map(lambda x: x['val_acc'])))
        full_record_with_hyperparams = test_records[index_of_max]
        # print(f"Hyperparameters for {full_record_with_hyperparams['args']['dataset']}, env {full_record_with_hyperparams['args']['test_envs']}:", full_record_with_hyperparams['hparams'])
        return test_records.map(self._step_acc).argmax('val_acc')

class LeaveOneOutSelectionMethod(SelectionMethod):
    """Picks (hparams, step) by leave-one-out cross validation."""
    name = "leave-one-domain-out cross-validation"

    @classmethod
    def _step_acc(self, records):
        """Return the {val_acc, test_acc} for a group of records corresponding
        to a single step."""
        test_records = get_test_records(records)
        if len(test_records) != 1:
            return None

        test_env = test_records[0]['args']['test_envs'][0]
        n_envs = 0
        for i in itertools.count():
            if f'env{i}_out_acc' not in records[0]:
                break
            n_envs += 1
        val_accs = np.zeros(n_envs) - 1
        for r in records.filter(lambda r: len(r['args']['test_envs']) == 2):
            val_env = (set(r['args']['test_envs']) - set([test_env])).pop()
            val_accs[val_env] = r['env{}_in_acc'.format(val_env)]
        val_accs = list(val_accs[:test_env]) + list(val_accs[test_env+1:])
        if any([v==-1 for v in val_accs]):
            return None
        val_acc = np.sum(val_accs) / (n_envs-1)
        return {
            'val_acc': val_acc,
            'test_acc': test_records[0]['env{}_in_acc'.format(test_env)]
        }

    @classmethod
    def run_acc(self, records):
        step_accs = records.group('step').map(lambda step, step_records:
            self._step_acc(step_records)
        ).filter_not_none()
        if len(step_accs):
            return step_accs.argmax('val_acc')
        else:
            return None


In [4]:
def format_mean(data, latex):
    """Given a list of datapoints, return a string describing their mean and
    standard error"""
    if len(data) == 0:
        return None, None, "X"
    mean = 100 * np.mean(list(data))
    err = 100 * np.std(list(data) / np.sqrt(len(data)))
    if latex:
        return mean, err, "{:.1f} $\\pm$ {:.1f}".format(mean, err)
    else:
        return mean, err, "{:.1f} +/- {:.1f}".format(mean, err)

def print_table(table, header_text, row_labels, col_labels, colwidth=10,
    latex=True):
    """Pretty-print a 2D array of data, optionally with row/col labels"""
    print("")

    if latex:
        num_cols = len(table[0])
        print("\\begin{center}")
        print("\\adjustbox{max width=\\textwidth}{%")
        print("\\begin{tabular}{l" + "c" * num_cols + "}")
        print("\\toprule")
    else:
        print("--------", header_text)

    for row, label in zip(table, row_labels):
        row.insert(0, label)

    if latex:
        col_labels = ["\\textbf{" + str(col_label).replace("%", "\\%") + "}"
            for col_label in col_labels]
    table.insert(0, col_labels)

    for r, row in enumerate(table):
        misc.print_row(row, colwidth=colwidth, latex=latex)
        if latex and r == 0:
            print("\\midrule")
    if latex:
        print("\\bottomrule")
        print("\\end{tabular}}")
        print("\\end{center}")

def print_results_tables(records, selection_method, latex):
    """Given all records, print a results table for each dataset."""
    grouped_records = reporting.get_grouped_records(records).map(lambda group:
        { **group, "sweep_acc": selection_method.sweep_acc(group["records"]) }
    ).filter(lambda g: g["sweep_acc"] is not None)

    # read algorithm names and sort (predefined order)
    alg_names = Q(records).select("args.algorithm").unique()
    alg_names = ([n for n in algorithms.ALGORITHMS if n in alg_names] +
        [n for n in alg_names if n not in algorithms.ALGORITHMS])

    # read dataset names and sort (lexicographic order)
    dataset_names = Q(records).select("args.dataset").unique().sorted()
    dataset_names = [d for d in datasets.DATASETS if d in dataset_names]

    for dataset in dataset_names:
        if latex:
            print()
            print("\\subsubsection{{{}}}".format(dataset))
        test_envs = range(datasets.num_environments(dataset))
        # breakpoint()
        table = [[None for _ in [*test_envs, "Avg"]] for _ in alg_names]
        for i, algorithm in enumerate(alg_names):
            means = []
            for j, test_env in enumerate(test_envs):
                trial_accs = (grouped_records
                    .filter_equals(
                        "dataset, algorithm, test_env",
                        (dataset, algorithm, test_env)
                    ).select("sweep_acc"))
                mean, err, table[i][j] = format_mean(trial_accs, latex)
                means.append(mean)
            if None in means:
                table[i][-1] = "X"
            else:
                table[i][-1] = "{:.1f}".format(sum(means) / len(means))

        col_labels = [
            "Algorithm",
            *datasets.get_dataset_class(dataset).ENVIRONMENTS,
            "Avg"
        ]
        header_text = (f"Dataset: {dataset}, "
            f"model selection method: {selection_method.name}")
        print_table(table, header_text, alg_names, list(col_labels),
            colwidth=20, latex=latex)

    # Print an "averages" table
    if latex:
        print()
        print("\\subsubsection{Averages}")

    table = [[None for _ in [*dataset_names, "Avg"]] for _ in alg_names]
    for i, algorithm in enumerate(alg_names):
        means = []
        for j, dataset in enumerate(dataset_names):
            trial_averages = (grouped_records
                .filter_equals("algorithm, dataset", (algorithm, dataset))
                .group("trial_seed")
                .map(lambda trial_seed, group:
                    group.select("sweep_acc").mean()
                )
            )
            mean, err, table[i][j] = format_mean(trial_averages, latex)
            means.append(mean)
        if None in means:
            table[i][-1] = "X"
        else:
            table[i][-1] = "{:.1f}".format(sum(means) / len(means))

    col_labels = ["Algorithm", *dataset_names, "Avg"]
    header_text = f"Averages, model selection method: {selection_method.name}"
    print_table(table, header_text, alg_names, col_labels, colwidth=25,
        latex=latex)

In [12]:
# input_dir = "./results_vits_3600_32"
input_dir = "./results_vits_terra_pacs"

# input_dir = "./results_vits_hessian_vlcs_terra_anneal_2500"
# input_dir = "./results_vits_hessian_VLCS"
# input_dir = "./results_vits_hessian_pacs_anneal_3600"
# input_dir = "./results_vits_hessian_vlcs_terra_anneal_2500"
# input_dir = "./results_vits_hessian_rescale" 
# input_dir = "./results_vits_hessian_vlcs_random2"
# input_dir = "./results_vits_hessian_vlcs_terra_anneal_2500"
# input_dir = "./results_vits_hessian_class_mnist" 
input_dir = "./results_vits_hessian_class_terra"
# input_dir = "./results_vits_hessian_bias_old"
input_dir = "./results_vits_hessian_bias"
input_dir = "./results_vits_hessian_real_bias" 
input_dir = "./results_hgp"
# ! python -m copy_32.py
# input_dir = "./results_vits_coral" 
# input_dir = "./results_vits_coral_MNIST"  #DONE


# input_dir = "./results_vits_hessian_MNIST_rescale"
# input_dir = "./results_vits_hessian_MNIST_rescale_sqrt"
# input_dir = "./results_vits_hessian_MNIST_rescale"
input_dir = "./results_vits_combined_bias"
# input_dir = "./results_resnet_mnist"
# input_dir = "./results_hgp_cma"
# input_dir = "./results_hess_diag"
input_dir = "./results_resnet"
# latex = True
latex = False
results_file = "results.tex" if latex else "results.txt"

records = reporting.load_records(input_dir)
# selection_methods = [
#     model_selection.IIDAccuracySelectionMethod,
#     model_selection.OracleSelectionMethod
# ]
selection_methods = [
    IIDAccuracySelectionMethod,
    OracleSelectionMethod,
]

for selection_method in selection_methods:
    print_results_tables(records, selection_method, latex)
    
    





-------- Dataset: VLCS, model selection method: training-domain validation set
Algorithm             C                     L                     S                     V                     Avg                  
ERM                   97.9 +/- 0.5          64.1 +/- 0.5          72.1 +/- 1.9          76.5 +/- 0.3          77.6                 

-------- Dataset: PACS, model selection method: training-domain validation set
Algorithm             A                     C                     P                     S                     Avg                  
ERM                   88.5 +/- 0.0          81.8 +/- 0.0          96.9 +/- 0.6          77.5 +/- 0.0          86.2                 

-------- Dataset: TerraIncognita, model selection method: training-domain validation set
Algorithm             L100                  L38                   L43                   L46                   Avg                  
ERM                   54.7 +/- 0.3          46.2 +/- 2.1          56.7 +/- 0.6          36

In [21]:
input_dir = "./results_vits_3600_32"
# input_dir = "./results_vits_terra_pacs"

# input_dir = "./results_vits_hessian_vlcs_terra_anneal_2500"
# input_dir = "./results_vits_hessian_VLCS"
# input_dir = "./results_vits_hessian_pacs_anneal_3600"
# input_dir = "./results_vits_hessian_vlcs_terra_anneal_2500"
# input_dir = "./results_vits_hessian_rescale" 
# input_dir = "./results_vits_hessian_vlcs_random2"
# input_dir = "./results_vits_hessian_vlcs_terra_anneal_2500"
# input_dir = "./results_vits_hessian_class" 
# input_dir = "./results_vits_hessian_class_terra"
input_dir = "./results_vits_combined_bias"
latex = False
results_file = "results.tex" if latex else "results.txt"

records = reporting.load_records(input_dir)
# selection_methods = [
#     model_selection.IIDAccuracySelectionMethod,
#     model_selection.OracleSelectionMethod
# ]
selection_methods = [
    IIDAccuracySelectionMethod,
    OracleSelectionMethod,
]

for selection_method in selection_methods:
    print_results_tables(records, selection_method, latex)
    
    




-------- Dataset: ColoredMNIST, model selection method: training-domain validation set
Algorithm             +90%                  +80%                  -90%                  Avg                  
ERM                   72.2 +/- 0.2          72.9 +/- 0.2          10.1 +/- 0.1          51.7                 
CORAL                 71.7 +/- 0.4          73.2 +/- 0.1          10.2 +/- 0.1          51.7                 
Fishr                 72.6 +/- 0.3          73.3 +/- 0.1          10.6 +/- 0.2          52.2                 
HessianAlignment      71.4 +/- 0.3          72.8 +/- 0.1          10.0 +/- 0.2          51.4                 

-------- Dataset: RotatedMNIST, model selection method: training-domain validation set
Algorithm             0                     15                    30                    45                    60                    75                    Avg                  
ERM                   95.3 +/- 0.2          98.6 +/- 0.1          99.1 +/- 0.1          98.9 +/- 0

In [10]:
 input_dir = "./results_vits_3600_32"
latex = False
results_file = "results.tex" if latex else "results.txt"

records = reporting.load_records(input_dir)
# selection_methods = [
#     model_selection.IIDAccuracySelectionMethod,
#     model_selection.OracleSelectionMethod
# ]
selection_methods = [
    IIDAccuracySelectionMethod,
    OracleSelectionMethod,
]

for selection_method in selection_methods:
    print_results_tables(records, selection_method, latex)
    
# a = selection_method.hparams_accs(records)
# grouped_records = reporting.get_grouped_records(records).map(lambda group:
#     { **group, "sweep_acc": selection_method.sweep_acc(group["records"]) }
# ).filter(lambda g: g["sweep_acc"] is not None)


-------- Dataset: ColoredMNIST, model selection method: training-domain validation set
Algorithm             +90%                  +80%                  -90%                  Avg                  
ERM                   X                     X                     X                     X                    
Fishr                 X                     X                     X                     X                    
HessianAlignment      72.4 +/- 0.2          73.2 +/- 0.1          10.0 +/- 0.2          51.9                 

-------- Dataset: RotatedMNIST, model selection method: training-domain validation set
Algorithm             0                     15                    30                    45                    60                    75                    Avg                  
ERM                   X                     X                     X                     X                     X                     X                     X                    
Fishr                 X         

In [11]:
result_dir = './results_vits_3600_32'

for exp in os.listdir(result_dir):
    if not os.path.isdir(os.path.join(result_dir, exp)) or len(os.listdir(os.path.join(result_dir, exp))) == 0:
        continue
    with open(os.path.join(result_dir,exp, "results.jsonl")) as f:
        first_line = f.readline()
        first_line = json.loads(first_line)
    # if first_line['args']['algorithm'] == 'HessianAlignment' and first_line['args']['dataset'] == 'PACS' and first_line['args']['test_envs'] == [3]:
    if first_line['args']['algorithm'] == 'Fishr' and first_line['args']['dataset'] == 'PACS' and first_line['args']['test_envs'] == [3] :
        # load the jsonl file into a df
        df_fishr = pd.read_json(os.path.join(result_dir,exp, "results.jsonl"), lines=True)
        # if first_line['args']['algorithm'] == 'HessianAlignment' and first_line['args']['dataset'] == 'PACS' and first_line['args']['test_envs'] == [3]:
    if first_line['args']['algorithm'] == 'HessianAlignment' and first_line['args']['dataset'] == 'PACS' and first_line['args']['test_envs'] == [3] and first_line['hparams']['penalty_anneal_iters'] == 3600:
        # load the jsonl file into a df
        df= pd.read_json(os.path.join(result_dir,exp, "results.jsonl"), lines=True)
        

In [723]:
#extract the hyperparameters from df.hparams and args from df.args
df['grad_alpha'] = df['hparams'].apply(lambda x: x['grad_alpha'])
df['hess_beta'] = df['hparams'].apply(lambda x: x['hess_beta'])
df['penalty_anneal_iters'] = df['hparams'].apply(lambda x: x['penalty_anneal_iters'])
df['test_envs'] = df['args'].apply(lambda x: x['test_envs'])


In [724]:
df_fishr['test_envs'] = df_fishr['args'].apply(lambda x: x['test_envs'])

In [231]:
combined_dir = 'results_hgp_cma'
cma_dir = 'results_vits_combined_bias'
hgp_dir = 'results_hgp'



if not os.path.exists(combined_dir):
    os.makedirs(combined_dir)

for exp in os.listdir(cma_dir):
    if not os.path.isdir(os.path.join(cma_dir, exp)) or len(os.listdir(os.path.join(cma_dir, exp))) == 0:
        continue
    with open(os.path.join(cma_dir, exp, "results.jsonl")) as f:
        first_line = f.readline()
        first_line = json.loads(first_line)
    if (first_line['args']['algorithm'] in ['HessianAlignment']):
        os.system(f"cp -r {os.path.join(cma_dir, exp)} {combined_dir}")


for exp in os.listdir(hgp_dir):
    if not os.path.isdir(os.path.join(hgp_dir, exp)) or len(os.listdir(os.path.join(hgp_dir, exp))) == 0:
        continue
    with open(os.path.join(hgp_dir, exp, "results.jsonl")) as f:
        first_line = f.readline()
        first_line = json.loads(first_line)
    if (first_line['args']['algorithm'] in ['HGP']):
        os.system(f"cp -r {os.path.join(hgp_dir, exp)} {combined_dir}")


# terr_dir = './results_vits_hessian_class_terra'